In [156]:
def r_type(opcode, func3, func7, rs1, rs2, rd):
    opcode = "{:07b}".format(opcode & 0b1111111)
    func3 = "{:03b}".format(func3 & 0b111)
    func7 = "{:07b}".format(func7 & 0b1111111)
    rs1 = "{:05b}".format(rs1 & 0b11111)
    rs2 = "{:05b}".format(rs2 & 0b11111)
    rd = "{:05b}".format(rd & 0b11111)

    res = func7 + rs2 + rs1 + func3 + rd + opcode
    return int(res, 2)

def i_type(opcode, func3, rs1, rd, imm):
    opcode = "{:07b}".format(opcode & 0b1111111)
    func3 = "{:03b}".format(func3 & 0b111)
    rs1 = "{:05b}".format(rs1 & 0b11111)
    rd = "{:05b}".format(rd & 0b11111)
    imm = "{:012b}".format(imm & 0b111111111111)

    res = imm + rs1 + func3 + rd + opcode
    return int(res, 2)

def s_type(opcode, func3, rs1, rs2, imm):
    opcode = "{:07b}".format(opcode & 0b1111111)
    func3 = "{:03b}".format(func3 & 0b111)
    rs1 = "{:05b}".format(rs1 & 0b11111)
    rs2 = "{:05b}".format(rs2 & 0b11111)
    imm = "{:012b}".format(imm & 0b111111111111)

    res = imm[0:7] + rs2 + rs1 + func3 + imm[7:12] + opcode
    return int(res, 2)

def b_type(opcode, func3, rs1, rs2, imm):
    opcode = "{:07b}".format(opcode & 0b1111111)
    func3 = "{:03b}".format(func3 & 0b111)
    rs1 = "{:05b}".format(rs1 & 0b11111)
    rs2 = "{:05b}".format(rs2 & 0b11111)
    imm = "{:013b}".format(imm & 0b1111111111111)

    res = imm[0] + imm[2:8] + rs2 + rs1 + func3 + imm[8:12] + imm[1] + opcode
    return int(res, 2)

def u_type(opcode, rd, imm):
    opcode = "{:07b}".format(opcode & 0b1111111)
    rd = "{:05b}".format(rd & 0b11111)
    imm = "{:020b}".format(imm & 0b11111111111111111111)

    res = imm + rd + opcode
    return int(res, 2)

def j_type(opcode, rd, imm):
    opcode = "{:07b}".format(opcode & 0b1111111)
    rd = "{:05b}".format(rd & 0b11111)
    imm = "{:020b}".format(imm & 0b11111111111111111111)

    res = imm[0] + imm[10:20] + imm[9] + imm[1:9] + rd + opcode
    return int(res, 2)

In [177]:
def immGen(instruction, immSel):
    ins = "{:032b}".format(instruction)
    if immSel == 0: # U-type, not SE
        return int(ins[0:20] + ("0" * 12), 2)
    elif immSel == 1: # J-type, SE
        imm = ins[0] + ins[12:20] + ins[11] + ins[1:11] + "0"
        if imm[0] == "1":
            imm = "1" * 11 + imm
        else:
            imm = "0" * 11 + imm
        return int(imm, 2)
    elif immSel == 2: # B-type, SE
        imm = ins[0] + ins[24] + ins[1:7] + ins[20:24] + "0"
        if imm[0] == "1":
            imm = "1" * 19 + imm
        else:
            imm = "0" * 19 + imm
        return int(imm, 2)
    elif immSel == 3: # I-type, not SE
        return int(ins[0:12], 2)
    elif immSel == 4: # I-type, SE
        imm = ins[0:12]
        if imm[0] == "1":
            imm = "1" * 20 + imm
        else:
            imm = "0" * 20 + imm
        return int(imm, 2)
    elif immSel == 5: # S-type, SE
        imm = ins[0:7] + ins[20:25]
        if imm[0] == "1":
            imm = "1" * 20 + imm
        else:
            imm = "0" * 20 + imm
        return int(imm, 2)


In [178]:
'''Read the instructions file'''
file = open("isa.txt", "r")
lines = file.readlines()

instructions = {}

for line in lines:
    line = line.split(",")
    ins_type = line[0]
    name = line[1]
    func7 = int(line[2], 2)
    func3 = int(line[3], 2)
    opcode = int(line[4], 2)

    instructions[name] = [ins_type, opcode, func3, func7]

file.close()

In [180]:
''' 
Generate testbench for the ImmGen module
'''
import random
rs1 = 6
rs2 = 8
rd = 23
imms = []
number_of_ims = 100

for i in range(number_of_ims):
    imms.append(random.randint(0, 2**32 - 1))

immSel = {"U_TYPE":0, "J_TYPE":1, "B_TYPE":2, "I_TYPE":3, "S_TYPE":5,}
to_write = []
for imm in imms:
    for name in instructions:
        ins_type = instructions[name][0]
        opcode = instructions[name][1]
        func3 = instructions[name][2]
        func7 = instructions[name][3]

        if ins_type == "U_TYPE":
            ins = u_type(opcode, rd, imm)
        elif ins_type == "J_TYPE":
            ins = j_type(opcode, rd, imm)
        elif ins_type == "B_TYPE":
            ins = b_type(opcode, func3, rs1, rs2, imm)
        elif ins_type == "I_TYPE":
            ins = i_type(opcode, func3, rs1, rd, imm)
        elif ins_type == "S_TYPE":
            ins = s_type(opcode, func3, rs1, rs2, imm)
        else:
            continue

        immGenerated = immGen(ins, immSel[ins_type])
        to_write.append("{:032b} {:03b} {:032b}".format(ins, immSel[ins_type], immGenerated))
        
file = open("immgen_tb.txt", "w")
file.write("\n".join(to_write))
file.close()


In [174]:
len("111111111111110110010110101001100")

33